In [1]:
library("readxl")

Warning message:
"package 'readxl' was built under R version 3.6.3"

In [2]:
library("lubridate")

Warning message:
"package 'lubridate' was built under R version 3.6.3"
Attaching package: 'lubridate'

The following objects are masked from 'package:base':

    date, intersect, setdiff, union



In [6]:
search()

[1] ".GlobalEnv"         "package:readxl"     "package:tseries"   
 [4] "package:xtable"     "package:quantmod"   "package:TTR"       
 [7] "package:reshape2"   "package:lubridate"  "package:scales"    
[10] "package:astsa"      "package:ggplot2"    "package:timeSeries"
[13] "package:timeDate"   "package:rugarch"    "package:parallel"  
[16] "package:xts"        "package:zoo"        "package:forecast"  
[19] "jupyter:irkernel"   "package:stats"      "package:graphics"  
[22] "package:grDevices"  "package:utils"      "package:datasets"  
[25] "package:methods"    "Autoloads"          "package:base"

In [3]:
source("helpers.R")

In [8]:
path <- "../data/data.xlsx"

In [9]:
df <- Dataframing(path)

In [10]:
df <- df[-c(210),]

In [11]:
df$Date <- as.Date(df$Date)

In [12]:
firstYields <- df[df$Date == "2022-02-11",]

In [11]:
firstYields

,Date,10Y,7Y,5Y,3Y,2Y,1Y,6M,3M
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
209,2022-02-11,1.941,1.94,1.856,1.728,1.505,1.029,0.692,0.359


#### Suponemos que: 
- $\alpha_1$ representa la tasa forward entre 0 y 3 meses, 
- $\alpha_2$ la tasa entre 3 meses y 1 año, 
- $\alpha_3$ la tasa entre 1 y 3 años,
- $\alpha_4$ la tasa entre 3 y 5 años ,
- $\alpha_5$ la tasa entre 5 y 10 años. 

In [12]:
today <- firstYields$Date

#### Además, para traer a valor presente usando una curva forward, se sigue el factor de descuento a un plazo T es: 

### $$ e^{-\int_{0}^{T} f(t) \,dt}$$

#### Definamos algunos objetos de utilidad. Cuántos cupones paga cada bono? Cómo ajustamos la tasa para cada pago?

In [13]:
alpha <- c(0.1,0.11,0.12,0.13,0.14)

In [14]:
alpha

[1] 0.10 0.11 0.12 0.13 0.14

In [15]:
today = firstYields$Date

In [16]:
threeMonthBond <- 100 + RateConverter(firstYields[["3M"]],3,today)*DiscountFactor(alpha, 3)
threeMonthBond
RateConverter(firstYields[["3M"]],3,today)
RateConverter(0.359,3,today)
RateConverter(firstYields[["1Y"]],12,today)


[1] 100.0866

[1] 0.08875278

[1] 0.08875278

[1] 1.043292

In [17]:
sixMonthBond <- 100 + RateConverter(firstYields[["6M"]], 6, today)*DiscountFactor(alpha, 6)

In [18]:
sixMonthBond

[1] 100.3301

In [19]:
oneYearBond <- 100 +  RateConverter(firstYields[["1Y"]], 6, today) *DiscountFactor(alpha, 6) + RateConverter(firstYields[["1Y"]], 12, today)*DiscountFactor(alpha, 12)

In [20]:
oneYearBond

[1] 101.4279

In [21]:
maturity<- 36

In [22]:
periods <- seq(6, maturity, 6)

In [23]:
threeMonthBond

[1] 100.0866

Recordemos que cada pago está dado por :  

$$\$100\times R \times \frac{ACT}{360}$$

In [19]:
singleBondPayment <- function(maturity, yield, alpha, today){
    return (RateConverter(yield, maturity,today)*DiscountFactor(alpha, maturity))
}

In [20]:
singleBondPayment(120,firstYields[["10Y"]], alpha, today)
firstYields[["10Y"]]

[1] 5.326155

[1] 1.941

In [21]:
yieldList <- list(firstYields)

In [22]:
nombres <- colnames(firstYields)

In [23]:
nombres <- nombres[-1]

In [24]:
nombres

[1] "10Y" "7Y"  "5Y"  "3Y"  "2Y"  "1Y"  "6M"  "3M"

In [25]:
nombres <- as.list(nombres)

In [26]:
names(nombres) <- c(120,84,60,36,24,12,6,3) 

In [27]:
BondValue <- function (today, maturity, yieldlist,alpha ){
    if (maturity == 3){
        return (100*DiscountFactor(alpha, maturity) + singleBondPayment(3,yieldlist[["3M"]], alpha, today))
        }
    setter <- as.character(maturity)
    periods <- seq(6, maturity, 6)
    bondSum <- sum(unlist(lapply(periods, function(x) singleBondPayment(x, yield = yieldlist[[nombres[[setter]]]],  alpha = alpha, today = today))))
    return(bondSum +100*DiscountFactor(alpha, maturity))
}



In [28]:
BondValue(today, 120, firstYields, alpha)

[1] 118.7522

In [29]:
maturities <- c(120,84,60,36,24,12,6,3) 

In [30]:
alpha <- c(0.00343939424789704, 0.0105322205923502, 0.0817121232717679, 0.159800322624318, 0.274481567907549)
bondValues <- list()
for (i in maturities){
    bondValues <- append(bondValues, BondValue(today, i, firstYields, alpha))
}

In [36]:
bondValues

[[1]]
[1] 100.9899

[[2]]
[1] 98.09315

[[3]]
[1] 100.8309

[[4]]
[1] 100.7474

[[5]]
[1] 98.57663

[[6]]
[1] 100.6777

[[7]]
[1] 99.99803

[[8]]
[1] 100.0027

In [31]:
error <-function(alpha){
    maturities <- c(3,6,12,24,36,60,84,120) 
    bondValues <- list()
    for (i in maturities){
        bondValues <- append(bondValues, BondValue(today, i, firstYields, alpha))
    }
    error <- sum((unlist(bondValues)-100)**2)
    return (error)
}

error(alpha)

[1] 8.350227

In [32]:

optim(alpha, error, lower = 0, upper= 1, method = "L-BFGS-B")

$par
[1] 0.003439394 0.010532221 0.081712123 0.159800323 0.274481568

$value
[1] 8.350227

$counts
function gradient 
      21       21 

$convergence
[1] 52

$message
[1] "ERROR: ABNORMAL_TERMINATION_IN_LNSRCH"

In [39]:
opt <- optim(alpha, error, lower = 0, upper= 1, method = "L-BFGS-B")

In [2]:
list(opt$par)

ERROR: Error in eval(expr, envir, enclos): objeto 'opt' no encontrado


Valoración de bono de 7 años

In [33]:
alpha <- c(0.00343939424789704, 0.0105322205923502, 0.0817121232717679, 0.159800322624318, 0.274481567907549)
pvnotional <- 100 - DiscountFactor(alpha[4], 84)*100
pvnotional
couponsum <- function (c){
pvcoupon <- DiscountFactor(alpha[2], 12)*RateConverter(c, 12,today) + DiscountFactor(alpha[2], 24)*RateConverter(c,24,today) +
 DiscountFactor(alpha[3], 36)*RateConverter(c, 36,today) +DiscountFactor(alpha[3], 48)*RateConverter(c, 48,today) +
 DiscountFactor(alpha[4], 60)*RateConverter(c, 60,today) + DiscountFactor(alpha[4], 72)*RateConverter(c, 72,today) +  
 DiscountFactor(alpha[4], 84)*RateConverter(c, 84,today) + DiscountFactor(alpha[4], 84)*RateConverter(c, 84,today)
 result <-pvnotional- pvcoupon
    if (result <0){
        return (500)
    }
    else {
        return (pvnotional- pvcoupon)
    }
    
}

opti <- optim(c,couponsum,lower = 0, upper=10,  method = "Brent")
opti


[1] 67.32638

$par
[1] 3.835352

$value
[1] 1.174788e-06

$counts
function gradient 
      NA       NA 

$convergence
[1] 0

$message
NULL

Matriz de curvas 3 años en el pasado

In [48]:

matAlphas <- list()
for ( r in 1:nrow(df)){ 
    firstYields <- df[r,]
    today <- firstYields$Date
    alpha <- c(0.00343939424789704, 0.0105322205923502, 0.0817121232717679, 0.159800322624318, 0.274481567907549)
    opt <- optim(alpha, error, lower = 0, upper= 1, method = "L-BFGS-B")
    matAlphas <- append(matAlphas, list(opt$par))
}

In [49]:
MatofAlphas <- matrix(unlist(matAlphas), ncol = 5, byrow = TRUE)

In [1]:
write.csv(MatofAlphas,"../data/sim.csv", row.names = FALSE)

ERROR: Error in is.data.frame(x): objeto 'MatofAlphas' no encontrado


In [4]:
MatofAlphas <- read.csv("../data/sim.csv")
i_alpha <- MatofAlphas[-nrow(MatofAlphas),]
f_alpha <- MatofAlphas[-1,]
delta_alpha <- f_alpha - i_alpha
cov_delta_a <- cov(delta_alpha)
MatofAlphas
cov_delta_a

V1,V2,V3,V4,V5
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.01469253,0.02332216,0.1072190,0.2294691,0.4065073
0.01469100,0.02455477,0.1072255,0.2269062,0.4057698
0.01490613,0.02528225,0.1075780,0.2279427,0.4036473
0.01564085,0.02441550,0.1094505,0.2287888,0.4081671
0.01664205,0.02500139,0.1098298,0.2266441,0.3961503
0.01613533,0.02459101,0.1081273,0.2228312,0.3942095
0.01536457,0.02662238,0.1066000,0.2187469,0.3804598
0.01546857,0.02551777,0.1076873,0.2208042,0.3855213
0.01601275,0.02596776,0.1123567,0.2269432,0.3872429


,V1,V2,V3,V4,V5
V1,6.029197e-07,4.079473e-07,1.073900e-06,2.077693e-06,3.446827e-06
V2,4.079473e-07,1.202395e-06,2.188503e-06,3.439836e-06,4.395852e-06
V3,1.073900e-06,2.188503e-06,1.418655e-05,2.654489e-05,3.529536e-05
V4,2.077693e-06,3.439836e-06,2.654489e-05,6.365702e-05,9.191198e-05
V5,3.446827e-06,4.395852e-06,3.529536e-05,9.191198e-05,2.068846e-04


In [5]:
eigen_delta <- eigen(cov_delta_a)
A <- eigen_delta$vectors 
sqrt <- eigen_delta$values**(1/2)
A 
sqrt
B <- t(t(A) * sqrt)
B

-0.01599859,-0.02116783,0.12298140,-0.47243402,0.872340326
-0.02232141,-0.07065955,0.34082166,-0.80218478,-0.484612381
-0.17436245,-0.44177253,0.80556336,0.34889055,0.061464005
-0.43761759,-0.76197566,-0.46488144,-0.10683784,-0.018837481
-0.88166568,0.46774915,0.06057301,0.01291292,-0.006365662


[1] 0.0161139505 0.0047960703 0.0016160575 0.0009371136 0.0006180125

-0.0002578005,-0.0001015224,1.987450e-04,-4.427243e-04,5.391172e-04
-0.0003596860,-0.0003388882,5.507874e-04,-7.517383e-04,-2.994965e-04
-0.0028096680,-0.0021187721,1.301837e-03,3.269501e-04,3.798552e-05
-0.0070517483,-0.0036544888,-7.512752e-04,-1.001192e-04,-1.164180e-05
-0.0142071172,0.0022433578,9.788948e-05,1.210087e-05,-3.934059e-06


In [6]:
Y <- MatofAlphas*A
Y

V1,V2,V3,V4,V5
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
-0.0002350598,0.0109089203,-0.0114550450,-0.040010790,0.138546506
-0.0003279239,0.0030197798,0.0013845938,-0.099298131,0.326873307
-0.0025990690,0.0086167371,0.0938446003,-0.200969233,-0.187648146
-0.0068447108,0.0196682315,-0.0530410466,-0.004842962,0.024723914
-0.0146727208,-0.0116226840,0.0067505812,-0.016014569,-0.187154878
-0.0003415500,0.0014895517,-0.0020368452,-0.098440708,-0.316228886
-0.0010856535,-0.0125773167,-0.0006785798,-0.166679847,0.132738812
-0.0068335882,-0.0204699682,-0.0017228454,0.103280960,-0.041188257
-0.0122013264,0.0090599068,-0.0025079588,0.027909787,0.005000436


In [7]:
W <- MatofAlphas*B
W

V1,V2,V3,V4,V5
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
-3.787742e-06,5.231995e-05,-1.073468e-05,-6.447319e-04,2.238991e-04
-5.284149e-06,4.880138e-06,1.297522e-06,-1.600085e-03,5.282461e-04
-4.188127e-05,1.392514e-05,5.799714e-05,-3.238408e-03,-3.032502e-04
-1.102953e-04,3.178499e-05,-3.278003e-05,-2.322719e-05,3.995527e-05
-2.364355e-04,-1.878293e-05,4.171944e-06,-7.680700e-05,-1.753854e-04
-1.638098e-06,2.407201e-06,-1.258796e-06,-4.721286e-04,-2.963424e-04
-5.206870e-06,-1.178637e-05,-4.193708e-07,-7.994083e-04,1.243913e-04
-3.277437e-05,-1.918269e-05,-2.776185e-05,4.953427e-04,-3.859807e-05
-5.851842e-05,8.490162e-06,-4.041312e-05,4.510382e-05,4.685976e-06


In [8]:
components <- princomp(delta_alpha)
summary(components)

Importance of components:
                           Comp.1      Comp.2      Comp.3       Comp.4
Standard deviation     0.01607517 0.004784527 0.001612168 0.0009348582
Proportion of Variance 0.90620972 0.080277851 0.009114614 0.0030648493
Cumulative Proportion  0.90620972 0.986487570 0.995602184 0.9986670336
                             Comp.5
Standard deviation     0.0006165251
Proportion of Variance 0.0013329664
Cumulative Proportion  1.0000000000

In [9]:
cum_variance <- cumsum(components$sdev^2 / sum(components$sdev^2))
best_components <- components$scores[,1:2]

In [10]:
best_components

,Comp.1,Comp.2
2,-0.0010133629,-0.0014999750
3,-0.0006067909,0.0020122557
4,0.0054039694,-0.0006699329
5,-0.0107083224,0.0042348343
6,-0.0029639914,-0.0027713995
7,-0.0134135066,0.0027898480
8,0.0062592469,-0.0003774793
9,0.0057672171,0.0059965248
10,0.0207177497,0.0035606578
11,0.0003116482,0.0003285832


In [11]:
librerias <- c("forecast","xts","rugarch","timeSeries","ggplot2","astsa","scales","lubridate","reshape2","quantmod","xtable","tseries")

In [12]:
if(length(setdiff(librerias, rownames(installed.packages()))) > 0){
  install.packages(setdiff(librerias, rownames(installed.packages())))}
invisible(sapply(librerias, require, character.only = TRUE,quietly = TRUE))

Warning message:
"package 'forecast' was built under R version 3.6.3"Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Warning message:
"package 'zoo' was built under R version 3.6.3"
Attaching package: 'zoo'

The following objects are masked from 'package:base':

    as.Date, as.Date.numeric

Warning message:
"package 'rugarch' was built under R version 3.6.3"
Attaching package: 'rugarch'

The following object is masked from 'package:stats':

    sigma

Warning message:
"package 'timeSeries' was built under R version 3.6.3"Warning message:
"package 'timeDate' was built under R version 3.6.2"
Attaching package: 'timeSeries'

The following object is masked from 'package:zoo':

    time<-


Attaching package: 'astsa'

The following object is masked from 'package:forecast':

    gas

Warning message:
"package 'scales' was built under R version 3.6.3"Warning messag

In [14]:
Arima_1 <- auto.arima(best_components[,1],stepwise = F,approximation = F)
Box.test(Arima_1$residuals)
Arima_2 <- auto.arima(best_components[,2],stepwise = F,approximation = F)
Box.test(Arima_2$residuals)


	Box-Pierce test

data:  Arima_1$residuals
X-squared = 0.0027584, df = 1, p-value = 0.9581



	Box-Pierce test

data:  Arima_2$residuals
X-squared = 0.022699, df = 1, p-value = 0.8802


Simulaciones al futuro

In [15]:
future <- 52
simulations <- 1000
Sim_component_1 <- replicate(expr = simulate(object = Arima_1,nsim = future),n = simulations)
Sim_component_2 <- replicate(expr = simulate(object = Arima_2,nsim = future),n = simulations)
for (sim in 1:simulations){
        nam <- paste("S", sim, sep = "_")
        assign(nam, cbind(Sim_component_1[,sim] , Sim_component_2[,sim]))
}

In [17]:
for (sim in 1:simulations){
        nam <- paste("S", sim, sep = "_")
        assign(nam, cbind(Sim_component_1[,sim] , Sim_component_2[,sim]))
}
S_1

,0.0358589962,6.034971e-04
,-0.0088991189,9.009222e-03
,-0.0058358649,9.508562e-03
,-0.0050305032,-3.087159e-03
,-0.0029199330,6.709224e-03
,0.0086950866,-3.074918e-03
,-0.0091219235,-3.132309e-03
,-0.0034793557,1.183359e-02
,-0.0245180468,-2.009493e-03
,0.0120605487,-3.134490e-03
,-0.0405953685,1.208458e-02
